In [1]:
import cv2
import numpy as np
from picamera.array import PiRGBArray
from picamera import PiCamera
import time
import RPi.GPIO as r

OSError: libmmal.so: cannot open shared object file: No such file or directory

In [ ]:
r.setmode(r.BCM)
r.setwarnings(False)
r.setup(18, r.OUT)
r.setup(23, r.OUT)
r.setup(24, r.OUT)
r.setup(25, r.OUT)

In [ ]:
# Initialize PiCamera and set resolution
camera = PiCamera()
camera.resolution = (640, 480)
camera.framerate = 32
rawCapture = PiRGBArray(camera, size=(640, 480))

In [ ]:
# Allow the camera to warm up
time.sleep(0.1)

In [ ]:
# Load Haar Cascade classifier for face detection
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [ ]:
# Define the known width of the face (in meters) and the focal length of the camera (in pixels)
KNOWN_WIDTH = 0.15 # meters
FOCAL_LENGTH = 600 # pixels

In [ ]:
# Divide the camera's field of view into four quadrants
width, height = camera.resolution
quadrant_width = int(width / 2)
quadrant_height = int(height / 2)